# Ship Detection Fine

In [1]:
import mmrotate
import torch
print(torch.__version__)

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


1.10.0+cu111


In [2]:
# 00.Reproduction_Test 폴더내의 configs 폴더안에서 아래와 같은 py 파일의 경로를 입력. 

config_file = '/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/configs/kfiou/roi_trans_kfiou_ln_r50_fpn_1x_dota_ms_rr_le90.py'

In [3]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

dataset_type = 'DOTADataset'
data_root = 'data/split_ms_dota1_0/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024)),
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le90'),
    dict(
        type='PolyRandomRotate',
        rotate_ratio=0.5,
        angles_range=180,
        auto_bound=False,
        rect_classes=[9, 11],
        version='le90'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
 

In [4]:
cfg.optimizer = dict(type='Adam', lr=1e-5, weight_decay=0.0001)
cfg.lr_config = dict(
    policy='CosineAnnealing',
    warmup='linear',
    warmup_iters=300,
    warmup_ratio=1.0 / 10,
    min_lr_ratio=1e-5)

In [5]:
# classes = ('Other Ship', 
#                  'Other Warship', 
#                  'Submarine', 
#                  'Other Aircraft Carrier', 
#                  'Ticonderoga',
#                  'Other Destroyer', 
#                  'Other Frigate', 
#                  'Patrol', 
#                  'Other Landing', 
#                  'Commander', 
#                  'Other Auxiliary Ship',
#                  'Other Merchant', 
#                  'Container Ship', 
#                  'RoRo', 
#                  'Cargo',
#                  'Barge', 
#                  'Tugboat', 
#                  'Ferry', 
#                  'Yacht', 
#                  'Sailboat', 
#                  'Fishing Vessel', 
#                  'Oil Tanker', 
#                  'Hovercraft',
#                  'Motorboat',
#                  'Dock')

In [6]:
for i in range(0,25):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [7]:
#--------------------------------------

# 앞서 학습된 resnet50 웨이트 파일을 00.Weights 폴더 경로를 참조하여 복사. 
# load pretrained resnet 
cfg.model.backbone.init_cfg.checkpoint = '/mnt/hdd/eric/.tmp_ipy/00.Weights/trained_weights_competition_period/resnet50_epoch_538_del_fc.pth'

# 앞서 학습된 dota 웨이트 파일을 아래와 같은 00.Weights 폴더 아래에 경로를 참조하여 복사
# load for transfer-learning
#cfg.load_from = "/mnt/hdd/eric/.tmp_ipy/00.Weights/trained_weights_competition_period/Transfer_dota_epoch_12.pth"
cfg.resume_from = "/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp_multi/epoch_12.pth"


# 학습된 웨이트가 위치하게 될 체크포인트 폴더 지정 
cfg.work_dir = f'/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp_multi'

# train 데이터셋 위치 => 앞서 merge했던 roas_split_1024_gap_128_merge_1_1.5_2 폴더로 지정. 
cfg.data_root = f"/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_with_Category"
cfg.data.train.data_root =f"/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_with_Category"
# roas_split_1024_gap_128_merge_1_1.5_2 폴더 아래에 annotation 폴더명  
cfg.data.train.ann_file="anns"
# roas_split_1024_gap_128_merge_1_1.5_2 폴더 아래에 image 폴더명 
cfg.data.train.img_prefix ="images_png"

# validation 데이터셋 위치
# 위와 같은 방법으로 oiltank_dataset/roas 폴더 아래에 roas_split_1024_gap_128_x1/train1024_1x 폴더 지정 
cfg.data.val.data_root = "/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_with_Category/valid/"
cfg.data.val.ann_file="anns"
cfg.data.val.img_prefix ="images_png"

# classes
cfg.model.roi_head.bbox_head[0].num_classes=25 
cfg.model.roi_head.bbox_head[1].num_classes=25



classes = ['Other_Ship', 
                 'Other_Warship', 
                 'Submarine', 
                 'Other_Aircraft_Carrier', 
                 'Ticonderoga',
                 'Other_Destroyer', 
                 'Other_Frigate', 
                 'Patrol', 
                 'Other_Landing', 
                 'Commander', 
                 'Other_Auxiliary_Ship',
                 'Other_Merchant', 
                 'Container_Ship', 
                 'RoRo', 
                 'Cargo',
                 'Barge', 
                 'Tugboat', 
                 'Ferry', 
                 'Yacht', 
                 'Sailboat', 
                 'Fishing_Vessel', 
                 'Oil_Tanker', 
                 'Hovercraft',
                 'Motorboat',
                 'Dock']

cfg.data.train.classes=classes
cfg.data.val.classes=classes

# epochs 
cfg.runner.max_epochs = 16

# batch 
cfg.data.samples_per_gpu = 2

# log interval
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 1
cfg.log_config.interval = 10

# gpu
cfg.gpu_ids = [0]
cfg.device='cuda'

# random seed 
cfg.seed=22


In [8]:
# 수정된 config 파일을 cfg.work_dir 내에 저장 

import os

print(cfg.work_dir)
if os.path.exists(os.path.join(cfg.work_dir)):
    pass
else:
    os.makedirs(os.path.join(cfg.work_dir))


if os.path.exists(os.path.join(cfg.work_dir,"ship_fine_tunning.py")):
    os.remove(os.path.join(cfg.work_dir,"ship_fine_tunning.py"))
    cfg.dump(os.path.join(cfg.work_dir,"ship_fine_tunning.py"))
    print("file removed")
else:
    cfg.dump(os.path.join(cfg.work_dir,"ship_fine_tunning.py"))

/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp_multi
file removed


In [9]:
"100000677.txt" in os.listdir("/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format/valid/anns")

True

In [10]:
!nvidia-smi

Thu Jan 11 16:38:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:3B:00.0 Off |                  N/A |
|  0%   31C    P8              32W / 350W |     14MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
%%bash 

export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb=256"
python tools/train.py /mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp_multi/ship_fine_tunning.py --seed 22

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/mmrotate/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/mmrotate/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable

Error while terminating subprocess (pid=781209): 
